In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0MB 5.4MB/s 
     |████████████████████████████████| 15.3MB 313kB/s 
     |████████████████████████████████| 6.5MB 41.3MB/s 


In [ ]:
!pip install keplergl

     |████████████████████████████████| 7.9MB 5.1MB/s 
  Created wheel for keplergl: filename=keplergl-0.2.2-py2.py3-none-any.whl size=15942747 sha256=e8640442a88fe3a55c468ced65e44350c9834a0a785cccce52a3c9218b8d0109
  Stored in directory: /root/.cache/pip/wheels/0e/78/c4/04ea0dbf6447ad708393751cd6ea453fb682f4c1c734f2247b
Successfully built keplergl


In [ ]:
import os
import glob

import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

from keplergl import KeplerGl

# Data processing


In [ ]:
vacsi_dep = 'df_vacc.csv'
#df_vacsi = pd.read_csv(vacsi_dep, delimiter =';')
df_vacsi = pd.read_csv(vacsi_dep)
df_vacsi = df_vacsi[1:]
df_vacsi['date'] = pd.to_datetime(df_vacsi['date'])
df_vacsi.head(10)

,Unnamed: 0,where,date,tot_vacc,daily,weekly,codelocation,cumul
1,1,01,2020-03-19,0.0,0.0,0.0,01,0.0
2,2,01,2020-03-20,0.0,0.0,0.0,01,0.0
3,3,01,2020-03-21,0.0,0.0,0.0,01,0.0
4,4,01,2020-03-22,0.0,0.0,0.0,01,0.0
5,5,01,2020-03-23,0.0,0.0,0.0,01,0.0
6,6,01,2020-03-24,0.0,0.0,0.0,01,0.0
7,7,01,2020-03-25,0.0,0.0,0.0,01,0.0
8,8,01,2020-03-26,0.0,0.0,0.0,01,0.0
9,9,01,2020-03-27,0.0,0.0,0.0,01,0.0
10,10,01,2020-03-28,0.0,0.0,0.0,01,0.0


In [ ]:
#day data

df_jour = df_vacsi['date'].drop_duplicates()
max = df_jour.max()

In [ ]:
max

Timestamp('2021-04-11 00:00:00')

In [ ]:
df_jour

1     2020-03-19
2     2020-03-20
3     2020-03-21
4     2020-03-22
5     2020-03-23
         ...    
386   2021-04-08
387   2021-04-09
388   2021-04-10
389   2021-04-11
390   2020-03-18
Name: date, Length: 390, dtype: datetime64[ns]

In [ ]:
#preprocessing

df_vacsi['code'] = df_vacsi['codelocation']

temp_df = dict()
for i in df_jour :
    temp_df[i] = df_vacsi.loc[ df_vacsi['date'] == i]
dict_df = dict()
for key, df in temp_df.items() :
    dict_df[key] = df.groupby('code').max().drop(['where','Unnamed: 0'], axis=1)
    
dict_df[max].head()

,date,tot_vacc,daily,weekly,codelocation,cumul
code,,,,,,
01,2021-04-11,85073.0,0.0,16156.0,01,85073.0
02,2021-04-11,79669.0,0.0,11366.0,02,79669.0
03,2021-04-11,68130.0,0.0,7628.0,03,68130.0
04,2021-04-11,31674.0,0.0,3466.0,04,31674.0
05,2021-04-11,28179.0,0.0,3775.0,05,28179.0


In [ ]:
geospatial_data  = 'departements-avec-outre-mer.geojson'
f = gpd.read_file(geospatial_data)
f = f.set_index('code')
f = f.sort_index(axis=0)

f.head()

,nom,geometry
code,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ..."
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


In [ ]:
dict_vaccin = dict()
for i in df_jour :
    dict_vaccin[i] = pd.concat([f,dict_df[i]], axis = 1)
dict_vaccin[max].head(1000)

,nom,geometry,date,tot_vacc,daily,weekly,codelocation,cumul
code,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2021-04-11,85073.0,0.0,16156.0,01,85073.0
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2021-04-11,79669.0,0.0,11366.0,02,79669.0
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2021-04-11,68130.0,0.0,7628.0,03,68130.0
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2021-04-11,31674.0,0.0,3466.0,04,31674.0
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2021-04-11,28179.0,0.0,3775.0,05,28179.0
...,...,...,...,...,...,...,...,...
971,Guadeloupe,"MULTIPOLYGON (((-61.79038 16.16682, -61.79164 ...",2021-04-11,11767.0,0.0,1271.0,971,11767.0
972,Martinique,"MULTIPOLYGON (((-60.86733 14.38867, -60.86802 ...",2021-04-11,23245.0,0.0,6221.0,972,23245.0
973,Guyane,"MULTIPOLYGON (((-53.87049 5.74494, -53.86464 5...",2021-04-11,12191.0,0.0,1850.0,973,12191.0


In [ ]:
min = df_jour.min()
dict_vaccin[min]

,nom,geometry,date,tot_vacc,daily,weekly,cumul
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",NaT,NaN,NaN,NaN,NaN
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-03-18,0.0,0.0,0.0,0.0
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-03-18,0.0,0.0,0.0,0.0
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-03-18,0.0,0.0,0.0,0.0
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-03-18,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
971,Guadeloupe,"MULTIPOLYGON (((-61.79038 16.16682, -61.79164 ...",2020-03-18,0.0,0.0,0.0,0.0
972,Martinique,"MULTIPOLYGON (((-60.86733 14.38867, -60.86802 ...",2020-03-18,0.0,0.0,0.0,0.0
973,Guyane,"MULTIPOLYGON (((-53.87049 5.74494, -53.86464 5...",2020-03-18,0.0,0.0,0.0,0.0
974,La Réunion,"MULTIPOLYGON (((55.56891 -21.37670, 55.56843 -...",2020-03-18,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np

test_ts_vac = pd.concat([dict_vaccin[min], dict_vaccin[max]])

test_ts_vac = test_ts_vac.dropna()

#test_ts_vac['date'] = test_ts_vac['date'].astype(str) +' '+ '00:00'




In [ ]:
test_ts_vac

,nom,geometry,date,tot_vacc,daily,weekly,cumul
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-03-18 00:00,0.0,0.0,0.0,0.0
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-03-18 00:00,0.0,0.0,0.0,0.0
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-03-18 00:00,0.0,0.0,0.0,0.0
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-03-18 00:00,0.0,0.0,0.0,0.0
06,Alpes-Maritimes,"MULTIPOLYGON (((7.06711 43.51365, 7.06665 43.5...",2020-03-18 00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
971,Guadeloupe,"MULTIPOLYGON (((-61.79038 16.16682, -61.79164 ...",2021-04-11 00:00,11767.0,0.0,1271.0,11767.0
972,Martinique,"MULTIPOLYGON (((-60.86733 14.38867, -60.86802 ...",2021-04-11 00:00,23245.0,0.0,6221.0,23245.0
973,Guyane,"MULTIPOLYGON (((-53.87049 5.74494, -53.86464 5...",2021-04-11 00:00,12191.0,0.0,1850.0,12191.0
974,La Réunion,"MULTIPOLYGON (((55.56891 -21.37670, 55.56843 -...",2021-04-11 00:00,45769.0,0.0,6350.0,45769.0


In [ ]:
test_ts_vac.to_csv('test_ts_vacc1.csv', index=True)

In [ ]:
ts_vacc = pd.concat(dict_vaccin, axis=0)
ts_vacc.sort_index(ascending=True)
ts_vacc['nom'] = ts_vacc['codelocation'] +' - '+ ts_vacc["nom"]
ts_vacc.tail(500)

nom  ...     cumul
           code                        ...          
2021-04-08 06    06 - Alpes-Maritimes  ...  235279.0
           07            07 - Ardèche  ...   54438.0
           08           08 - Ardennes  ...   45271.0
           09             09 - Ariège  ...   28203.0
           10               10 - Aube  ...   51327.0
...                               ...  ...       ...
2020-03-18 971       971 - Guadeloupe  ...       0.0
           972       972 - Martinique  ...       0.0
           973           973 - Guyane  ...       0.0
           974       974 - La Réunion  ...       0.0
           976          976 - Mayotte  ...       0.0

[500 rows x 8 columns]

In [ ]:
ts_vacc = ts_vacc.dropna()
#ts_vacc['date'] = ts_vacc['date'].astype(str) +' '+ '00:00'
ts_vacc.tail(500)

nom  ...     cumul
           code                        ...          
2021-04-08 05       05 - Hautes-Alpes  ...   26620.0
           06    06 - Alpes-Maritimes  ...  235279.0
           07            07 - Ardèche  ...   54438.0
           08           08 - Ardennes  ...   45271.0
           09             09 - Ariège  ...   28203.0
...                               ...  ...       ...
2020-03-18 971       971 - Guadeloupe  ...       0.0
           972       972 - Martinique  ...       0.0
           973           973 - Guyane  ...       0.0
           974       974 - La Réunion  ...       0.0
           976          976 - Mayotte  ...       0.0

[500 rows x 8 columns]

In [ ]:
tvac = ts_vacc[29492:-100]

In [ ]:
tvac = tvac.drop(["geometry"],axis=1)

KeyError: ignored

In [ ]:
tvac = tvac.drop(["codelocation"],axis=1)
tvac

KeyError: ignored

In [ ]:
tvac

nom       date  ...  weekly    cumul
           code                                           ...                 
2021-01-05 01                        01 - Ain 2021-01-05  ...     0.0      0.0
           02                      02 - Aisne 2021-01-05  ...     0.0      0.0
           03                     03 - Allier 2021-01-05  ...     0.0      0.0
           04    04 - Alpes-de-Haute-Provence 2021-01-05  ...     4.0      4.0
           05               05 - Hautes-Alpes 2021-01-05  ...     6.0      8.0
...                                       ...        ...  ...     ...      ...
2021-04-11 971               971 - Guadeloupe 2021-04-11  ...  1271.0  11767.0
           972               972 - Martinique 2021-04-11  ...  6221.0  23245.0
           973                   973 - Guyane 2021-04-11  ...  1850.0  12191.0
           974               974 - La Réunion 2021-04-11  ...  6350.0  45769.0
           976                  976 - Mayotte 2021-04-11  ...   867.0  11005.0

[9797 rows x 6 columns]

In [ ]:
tvac.to_csv('vac_code_nom_dep.csv', index= True)

In [ ]:
tvac_light = tvac[7000:]
tvac_light

nom  ...     cumul
           code                   ...          
2021-03-20 34            Hérault  ...  115611.0
           35    Ille-et-Vilaine  ...   97074.0
           36              Indre  ...   26683.0
           37     Indre-et-Loire  ...   55094.0
           38              Isère  ...  118898.0
...                          ...  ...       ...
2021-04-11 971        Guadeloupe  ...   11767.0
           972        Martinique  ...   23245.0
           973            Guyane  ...   12191.0
           974        La Réunion  ...   45769.0
           976           Mayotte  ...   11005.0

[2289 rows x 7 columns]

In [ ]:
tvac_Xlight = tvac_light[1500:]
tvac_Xlight

nom  ...     cumul
           code                 ...          
2021-04-04 21        Côte-d'Or  ...   86870.0
           22    Côtes-d'Armor  ...  112305.0
           23           Creuse  ...   21104.0
           24         Dordogne  ...   63341.0
           25            Doubs  ...   72411.0
...                        ...  ...       ...
2021-04-11 971      Guadeloupe  ...   11767.0
           972      Martinique  ...   23245.0
           973          Guyane  ...   12191.0
           974      La Réunion  ...   45769.0
           976         Mayotte  ...   11005.0

[789 rows x 7 columns]

In [ ]:
tvac_Xlight.to_csv('ts_vacc_LX1.csv', index= True)

In [ ]:
tvac_light.to_csv('ts_vacc_L.csv', index= True)

In [ ]:
map_vaccin_2D = KeplerGl(height=500, config=config)
map_vaccin_2D.add_data(dict_vaccin[max], name ='vaccin')
map_vaccin_2D

NameError: ignored

In [ ]:
config_2D = map_coronavirus_2D.config

NameError: ignored

In [ ]:
# Save map_1 config to a file
with open('config_2D.py', 'w') as f:
    f.write('config = {}'.format(config_2D))